In [18]:
import pandas as pd
import numpy as np


import re
import string

#NLTK
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

#Date-time
from datetime import datetime
import dateutil.parser as dparser


In [19]:
trump_raw = pd.read_csv("D:\Codes\PROJECT 2\hashtag_donaldtrump.csv",
                 lineterminator='\n')

biden_raw = pd.read_csv("D:\Codes\PROJECT 2\hashtag_joebiden.csv",
                 lineterminator='\n')

#trump_raw = pd.read_csv("C:/Users/ariver10/Downloads/archive/hashtag_donaldtrump.csv",
#                 lineterminator='\n')

#biden_raw = pd.read_csv("C:/Users/ariver10/Downloads/archive/hashtag_joebiden.csv",
#                 lineterminator='\n')

In [20]:
#Removes columns of data with very high Na values. 
trump=trump_raw.drop(['user_description','long','lat','city','continent',
'state','state_code'], axis=1)

#remove rows of data with less NaN values
trump=trump.dropna(subset=['user_name','source','user_location'])

print(f"This data has (rows x col){trump.shape}")
print(f"total NaN values in this dataFrame: {trump.isna().sum().sum()}")


This data has (rows x col)(675765, 14)
total NaN values in this dataFrame: 233165


In [21]:
#Removes columns of data with very high Na values. 
biden=biden_raw.drop(['user_description','long','lat','city','continent',
'state','state_code'], axis=1)

#remove rows of data with less NaN values
biden=biden.dropna(subset=['user_name','source','user_location'])

print(f"This data has (rows x col){biden.shape}")
print(f"total NaN values in this dataFrame: {biden.isna().sum().sum()}")

This data has (rows x col)(542952, 14)
total NaN values in this dataFrame: 189262


In [22]:
#Add ac column of T and a column of B in trump and biden respectively to easily be able to identify them
trump['candidate']=0
biden['candidate']=1
#full_df=biden.append(trump,ignore_index=True)
#full_df.shape


In [23]:
#CHECK OVER THIS FOR DATASETS WE DON'T NEED
trump_raw['both'] = trump_raw.tweet_id.isin(biden_raw.tweet_id).astype(int) #makes column for both candidates hashtagged
biden_raw['both'] = trump_raw.tweet_id.isin(biden_raw.tweet_id).astype(int)
trump_raw['hash'] = np.where((trump_raw['both'] == 0), 'Trump', 'Both') #makes categorical column 
biden_raw['hash'] = np.where((biden_raw['both'] == 0), 'Biden', 'Both')

full_df = trump_raw.append(biden_raw) #shape = (1522909, 21) #appends datasets together

tweets_loc = trump.append(biden) #adds trump and biden hashtag data into a single dataframe shape = (1747805, 21)

tweets_loc = tweets_loc.drop_duplicates(subset = 'tweet_id') #drops overlapping tweets (hashtag for both Trump and Biden) shape = (1059879, 21)

US = tweets_loc['country'] == 'United States of America' #selects only USA tweets

tweets_USA = tweets_loc[US] #shape = (301341, 21)


tweets_loc=tweets_loc.drop(['country'], axis=1).reset_index() #drops extra columns we wont use shape = (301341, 10)
tweets_USA=tweets_USA.drop(['country'], axis=1).reset_index()

## Preprocessing the tweets

In [24]:
processed_tweets = pd.read_csv("D:\Codes\PROJECT 2\processed_tweets.csv")

### ATTENTION: (This cell creates the processed.csv file. If you have the processed.csv file imported you donot need to run this part of the notebook. let it stay as a markdown cell)

In [34]:
# Processing the tweets
def process_tweet(tweet):
 
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove twitter abbreviations
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean



    ### Applying the above made function on all of the data set. (This is a very resource intensive task. Run at your own risk)

Tw={}    # Inititalizing a new dictionary to store the values in
    # A seperate iterator to avoid a double for loop
for i, tweet in enumerate(tweets_loc['tweet']):
    Tw[i]=process_tweet(tweet)
    

s = pd.Series(Tw)
s.to_csv(r"D:\Codes\PROJECT 2\tweets_loc.csv")

In [25]:
processed_tweets.head()
processed_tweets=processed_tweets.iloc[:,-1]

In [26]:
tweets_loc['tweet']=processed_tweets
tweets_loc.head()


,index,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_join_date,user_followers_count,user_location,collected_at,candidate
0,0,2020-10-15 00:00:01,1.316529e+18,"['eleccion', '2020', 'en', 'florida', 'joebide...",0.0,0.0,TweetDeck,3.606665e+08,El Sol Latino News,elsollatinonews,2011-08-23 15:33:45,1860.0,"Philadelphia, PA / Miami, FL",2020-10-21 00:00:00,0
1,2,2020-10-15 00:00:02,1.316529e+18,"['trump', 'student', 'use', 'hear', 'year', 't...",2.0,1.0,Twitter Web App,8.436472e+06,snarke,snarke,2007-08-26 05:56:11,1185.0,Portland,2020-10-21 00:00:00.746433060,0
2,4,2020-10-15 00:00:08,1.316529e+18,"['get', 'tie', 'get', 'tie', 'trump', '‘', 'ra...",4.0,3.0,Twitter for iPhone,4.741380e+07,Rana Abtar - رنا أبتر,Ranaabtar,2009-06-15 19:05:35,5393.0,Washington DC,2020-10-21 00:00:01.492866121,0
3,5,2020-10-15 00:00:17,1.316529e+18,"['15', 'minut', 'long', 'time', 'ago', 'omaros...",2.0,0.0,Twitter for Android,1.138416e+09,Farris Flagg,FarrisFlagg,2013-02-01 01:37:38,2363.0,"Perris,California",2020-10-21 00:00:01.866082651,0
4,6,2020-10-15 00:00:17,1.316529e+18,"['glad', 'u', 'got', 'hous', 'dick', 'trump', ...",0.0,0.0,Twitter for iPhone,7.674018e+17,Michael Wilson,wilsonfire9,2016-08-21 16:43:51,75.0,"Powell, TN",2020-10-21 00:00:02.239299182,0


In [27]:
x=tweets_loc.iloc[:,:-1]
y=tweets_loc.iloc[:,-1]
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2) # 80%: train 20%: test


### The purpose now is to create a dictionary where the keys are worlds in a tuple (word, label/sentiment) and the values are their corresponding frequencies

In [35]:
def count_tweets(result, tweets, ys):

    for y, tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            pair = (word,y)

            # if the key exists in the dictionary, increment the count
            if pair in result:
                result[pair] += 1

            # else, if the key is new, add it to the dictionary and set the count to 1
            else:
                result[pair] = 1

    return result


In [44]:
## A very resource intensive task output. Is saved as a .csv file named freqs and imported seperately
### Build the freqs dictionary for later uses

freqs = count_tweets({}, x_train['tweet'], y_train)
#freqs = pd.Series(freqs)
#freqs.to_csv(r"D:\Codes\PROJECT 2\freqs.csv")

In [45]:
freqs

{('feel', 0): 4077,
 ('hillaryclinton', 0): 556,
 ('put', 0): 4671,
 ('effort', 0): 861,
 ('wisconsin', 0): 4099,
 ('2016', 0): 6032,
 ('tonight', 0): 2961,
 ('would', 0): 14384,
 ('talk', 0): 5417,
 ('either', 0): 1938,
 ('trump', 0): 577885,
 ('biden', 0): 139136,
 ('joebiden', 1): 136299,
 ('vermeintlich', 0): 14,
 ('rebellion', 0): 22,
 ('gegen', 0): 1050,
 ('da', 0): 12148,
 ('system', 0): 1869,
 ('unbeliev', 1): 177,
 ('close', 1): 1968,
 ('us', 1): 13818,
 ('presidenti', 1): 3669,
 ('elect', 1): 66918,
 ('biden', 1): 224874,
 ('landslid', 1): 747,
 ('moral', 1): 409,
 ('alon', 1): 384,
 ('american', 1): 9949,
 ('peopl', 1): 11628,
 ('vote', 1): 52510,
 ('trump', 1): 40675,
 ('truli', 1): 615,
 ('hideou', 1): 18,
 ('bigot', 1): 93,
 ('elect', 0): 129177,
 ('2020', 0): 274476,
 ('electionnight', 0): 11958,
 ('electionday', 0): 23232,
 ('tuesdayvib', 0): 137,
 ('bidenharri', 0): 23985,
 ('great', 1): 4471,
 ('news', 1): 4772,
 ('46th', 1): 3284,
 ('presid', 1): 35148,
 ('first', 1)

freqs=pd.read_csv("D:\Codes\PROJECT 2\\freqs.csv")

### Training Naive Bayes


In [56]:
def lookup(freqs, word, label):

    n = 0  # freqs.get((word, label), 0)

    pair = (word, label)
    if (pair in freqs):
        n = freqs[pair]

    return n

In [57]:

def train_naive_bayes(freqs, train_x, train_y):
    
    loglikelihood = {}
    logprior = 0

   
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    N_pos = N_neg = 0
    for pair in freqs.keys():
      
        if pair[1] > 0:

            N_pos += freqs[pair]

        else:

            N_neg += freqs[pair]

    D = len(train_y)

    D_pos = (len(list(filter(lambda x: x > 0, train_y))))

    
    D_neg = (len(list(filter(lambda x: x <= 0, train_y))))

    #  logprior
    logprior = np.log(D_pos) - np.log(D_neg)

    
    for word in vocab:
        
        freq_pos = lookup(freqs,word,1)
        freq_neg = lookup(freqs,word,0)

        
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        
        loglikelihood[word] = np.log(p_w_pos/p_w_neg)


    return logprior, loglikelihood

In [58]:
logprior, loglikelihood = train_naive_bayes(freqs, x_train, y_train)
print(logprior)
print(len(loglikelihood))

-0.5607084780959823
450610


## Testing Naive Bayes

In [59]:

def naive_bayes_predict(tweet, logprior, loglikelihood):
  
    
    word_l = process_tweet(tweet)

    p = 0

    p += logprior

    for word in word_l:

        
        if word in loglikelihood:
            
            p += loglikelihood[word]

   

    return p

In [77]:


# Experiment with your own tweet.
my_tweet = 'vote for trump'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print('The expected output is', p)

The expected output is -2.3539346311724585


In [67]:

def test_naive_bayes(test_x, test_y, logprior, loglikelihood):

    accuracy = 0  

  
    y_hats = []
    for tweet in test_x:
      
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
          
            y_hat_i = 1
        else:
            
            y_hat_i = 0

       
        y_hats.append(y_hat_i)

  
    error = np.mean(np.absolute(y_hats-test_y))
    
    accuracy = 1-error

    return accuracy

In [84]:
test_naive_bayes(x_test['tweet'], y_test, logprior, loglikelihood)


0.879623164886591

In [85]:
print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(x_test['tweet'], y_test, logprior, loglikelihood)))

Naive Bayes accuracy = 0.8796
